- consider only item features
- modify functions for predictions of cold users
- add autox (remove directory after execution)
- try fasttext, but not succeed

In [1]:
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

Cloning into 'autox'...
remote: Enumerating objects: 2098, done.
remote: Counting objects: 100% (2098/2098), done.
remote: Compressing objects: 100% (1437/1437), done.
remote: Total 2098 (delta 1345), reused 1357 (delta 611), pack-reused 0
Receiving objects: 100% (2098/2098), 9.69 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (1345/1345), done.
Processing ./autox
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 1.3 MB 785 kB/s            
  Created wheel for automl-x: filename=automl_x-0.2.1-py3-none-any.whl size=4724379 sha256=64285aca3552b456e56675db1e95434873376f41cb02321fdab1fe37aef8f388
  Stored in directory: /tmp/pip-ephem-wheel-cache-q0cd7rer/wheels/d8/d0/3f/17c06e69ec61f9c250a3274a013cc8a24b47965ec33f40c73d
Successfully built automl-x


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import implicit
import pickle
from autox.autox_competition.feature_engineer import fe_ima2vec
import fasttext 
import fasttext.util

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

In [3]:
DATA_DIR = "/kaggle/input/h-and-m-personalized-fashion-recommendations/"
df = pd.read_csv(DATA_DIR + "transactions_train.csv")
article_df = pd.read_csv(DATA_DIR + "articles.csv")
customer_df = pd.read_csv(DATA_DIR + "customers.csv")
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [4]:
df["t_dat"] = pd.to_datetime(df["t_dat"])

date_week_df = df.drop_duplicates("t_dat")[["t_dat"]].reset_index(drop=True)
date_week_df['week_no'] = (date_week_df['t_dat'] + pd.DateOffset(days=5)).dt.week
date_week_df["week_no"] = date_week_df["week_no"].diff(1)
date_week_df["week_no"].fillna(0, inplace=True)
date_week_df["week_no"] = date_week_df["week_no"] != 0
date_week_df["week_no"] = date_week_df["week_no"].cumsum()

df = pd.merge(df, date_week_df, on="t_dat", how="left")

df.sort_values(['t_dat', 'customer_id'], inplace=True)

test_week = df.week_no.max()
last_ts = df['t_dat'].max()

# autox

In [5]:
articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', dtype={'article_id': str})
articles['sub_path'] = articles['article_id'].apply(lambda x: str(x)[:3])
articles = articles[["article_id", "sub_path", "detail_desc"]]

In [6]:
fe_img = fe_ima2vec(articles, 
                    img_path='../input/h-and-m-personalized-fashion-recommendations/images', 
                    img_col='article_id', 
                    filename_extension='jpg', 
                    sub_path = 'sub_path'
                   )

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1091)>

In [ ]:
fe_img.index = articles.article_id
fe_img.to_csv('autox_features.csv', index=None)

In [ ]:
!rm -rf ./autox
del articles, fe_img

# fasttext

In [ ]:
# https://fasttext.cc/docs/en/crawl-vectors.html
# https://www.kaggle.com/code/mschumacher/using-fasttext-models-for-robust-embeddings/notebook


#fasttext.util.download_model('en', if_exists='ignore') # English
#ft = fasttext.load_model('cc.en.300.bin')
#fasttext.util.reduce_model(ft, 100)

def text_to_vector(text):
    """
    Given a string, normalizes it, then splits it into words and finally converts
    it to a sequence of word vectors.
    """
    words = text.split()
    window = words[-window_length:]
    
    x = np.zeros((window_length, n_features))

    for i, word in enumerate(window):
        x[i, :] = ft.get_word_vector(word).astype('float32')

    return x

def df_to_data(df):
    """
    Convert a given dataframe to a dataset of inputs for the NN.
    """
    x = np.zeros((len(df), window_length, n_features), dtype='float32')

    for i, comment in enumerate(df['detail_desc'].values):
        x[i, :] = text_to_vector(comment)

    return x

#detail_desc_vec = df_to_data(articles)

# features by abstraction block

In [ ]:
df = df[df.week_no > df.week_no.max() - 50].reset_index(drop=True)

In [ ]:
article_columns = ["garment_group_no",
                "product_code", "product_type_no", 
                "graphical_appearance_no", "colour_group_code", "perceived_colour_value_id",
                "perceived_colour_master_id", "department_no", "index_group_no", 
                "section_no"]

df = pd.merge(df, article_df[["article_id"] + article_columns], on='article_id', how='left')

In [ ]:
# customerごとに何か購入したweek
customer2weeks = df.groupby('customer_id')['week_no'].unique()

# customerごとに学習データの特徴量となるweekとラベルとなるweekのペアを作る。
customer2weekpairs = {}

for c_id, weeks in customer2weeks.items():
    customer2weekpairs[c_id] = {}
    for i in range(weeks.shape[0]-1):
        customer2weekpairs[c_id][weeks[i]] = weeks[i+1]
    customer2weekpairs[c_id][weeks[-1]] = test_week

In [ ]:
def join(df):
    x = [str(e) for e in list(df)]
    return " ".join(x)

def label_prev_week(input_df):
    weeks = []
    for i, (c_id, week) in enumerate(zip(input_df['customer_id'], input_df['week_no'])):
        weeks.append(customer2weekpairs[c_id][week])
    input_df.week_no=weeks
    return input_df

In [ ]:
class AbstractBaseBlock:
    def fit(self, input_df, y=None):
        return self.transform(input_df)

    def transform(self, input_df):
        raise NotImplementedError()
        
class UserHistoryEveryBlock(AbstractBaseBlock):
    """ユーザーの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['customer_id', 'week_no'])[self.item].agg(["sum", "std", "max", "min"]).add_prefix("u_prev_"+self.item+"_").reset_index()
        out = label_prev_week(out)
        return self.transform(out)

    def transform(self, input_df):
        return input_df
    
class UserHistoryVectorEveryBlock(AbstractBaseBlock):
    """ユーザーの文字履歴を週間で集計しベクトル化した特徴量を付与する block 累積ではない"""
    def __init__(self, item, n_components = 5):
        self.item = item
        self.n_components = n_components
        
    def fit(self, input_df, y=None):
        docs = input_df.groupby(["customer_id", "week_no"])[self.item].apply(join)
        max_features = int(input_df[self.item].nunique() * 0.8)
        tv = TfidfVectorizer(max_features=max_features)
        X = tv.fit_transform(docs)
        
        svd = TruncatedSVD(n_components=self.n_components, random_state=0)
        X = svd.fit_transform(X)
        result = pd.DataFrame(X, columns=[f"u_{self.item}_{i}" for i in range(self.n_components)])
        result.index = docs.index
        result = result.reset_index()
        result = label_prev_week(result)
        return self.transform(result)
  
    def transform(self, input_df):
        return input_df  

class ItemHistoryEveryBlock(AbstractBaseBlock):
    """アイテムの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['week_no'])[self.item].value_counts()
        out = out.to_frame("sale_count").reset_index()
        return self.transform(out)

    def transform(self, input_df):
        return input_df
    
class ItemHistorySumBlock(AbstractBaseBlock):
    """アイテムの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['week_no', self.item])['price'].agg(["sum"]).add_prefix("sale_").reset_index()
        return self.transform(out)

    def transform(self, input_df):
        return input_df

In [ ]:
# user_features
feature_blocks = [
#    UserHistoryEveryBlock("price"),
    UserHistoryVectorEveryBlock("section_no"),
    UserHistoryVectorEveryBlock("garment_group_no"),
]

for i, block in enumerate(feature_blocks):
    if i == 0:
        user_features_df = block.fit(df)
    else:
        out_i = block.fit(df)
        user_features_df = pd.merge(user_features_df, out_i, on = ["customer_id", "week_no"], how="left")
        
user_features_df.to_csv("user_features.csv", index=False)
del user_features_df

In [ ]:
# item_features
article_count = ItemHistoryEveryBlock("article_id").fit(df)
article_count.to_csv("article_count.csv", index=False)

article_sale = ItemHistorySumBlock("article_id").fit(df)
article_sale.to_csv("article_sale.csv", index=False)